In [ ]:
# Для машинного обучения
pip install transformers
pip install torch

# Для анализа
pip install seaborn
pip install statsmodels

In [1]:
# Для сбора
import requests
import json
import random
import time

In [2]:
# Для машинного обучения
import torch

In [2]:
# Для анализа
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats

In [ ]:
# Функция - информация о продуктах категории
def get_products(shardKey, query, rawQuery):
    headers = {
        'Host': 'catalog.wb.ru',
        'Connection': 'close',
        # 'Accept-Encoding': 'gzip, deflate',
        'User-Agent': 'okhttp/4.10.0',
    }

    response = requests.get(
        f'SOME_API_LINK_FOR_SITE_{shardKey}/{query}&{rawQuery}',
        headers=headers,
        verify=False,
    )    
    return response

In [ ]:
# Функция - информация о количестве продаж продукта
def show_buys(product_id):
    headers = {
        'Host': 'Site_Host',
        'Connection': 'Keep-Alive',
        # 'Accept-Encoding': 'gzip, deflate',
        'User-Agent': 'okhttp/4.10.0',
    }

    params = {'nm': product_id}

    response = requests.get('SOME_API_LINK_FOR_SITE_', params=params, headers=headers, verify=False)
    return response.json()[0]["qnt"]

In [4]:
# Функция - информация об отзывах продукта
def get_about_product(root_id):
    headers = {
        'Host': 'Site_Host',
        'Connection': 'close',
        # 'Accept-Encoding': 'gzip, deflate',
        'User-Agent': 'okhttp/4.10.0',
    }

    response = requests.get(f'SOME_API_LINK_FOR_SITE_{root_id}', headers=headers, verify=False)
    return response

In [ ]:
all_categories = json.load(open("all_categories.json", encoding="utf-8"))

In [ ]:
# Запись необходимых для функции get_products() параметров и названия категорий и подкатегорий
parameters_list = []
for category in all_categories['data']['catalog']:
    category_name = category['name']
    if 'childNodes' in category:
        for subcategory in category['childNodes']:
            subcategory_name = subcategory['name']
            if "shardKey" in subcategory and "query" in subcategory and "rawQuery" in subcategory:
                parameters_list.append({"category_name": category_name,
                                        "subcategory_name": subcategory_name,
                                        "shardKey": subcategory["shardKey"],
                                        "query": subcategory["query"],
                                        "rawQuery": subcategory["rawQuery"]})
    else:
        if "shardKey" in category and "query" in category and "rawQuery" in category:
            parameters_list.append({"category_name": category_name,
                                    "shardKey": category["shardKey"],
                                    "query": category["query"],
                                    "rawQuery": category["rawQuery"]})

In [ ]:
# Выгрузка информации о продуктах
product_number = 0
while j < len(parameters_list):
    shardKey = parameters_list[j]["shardKey"]
    query = parameters_list[j]["query"]
    rawQuery = parameters_list[j]["rawQuery"]
    print("="*15)
    print(f"{j}) Отправка запроса")
    time.sleep(1)
    products_list = get_products(shardKey, query, rawQuery).json()['data']['products']
    
    for i in range(len(products_list)):
        data[str(product_number)] = products_list[i]
        if "category_name" in parameters_list[i]:
            data[str(product_number)]["category_name"] = parameters_list[i]["category_name"]
        if "subcategory_name" in parameters_list[i]:
            data[str(product_number)]["subcategory_name"] = parameters_list[i]["subcategory_name"]
        product_number += 1
    
    j += 1

In [ ]:
# Всего получилось продуктов
print(len(data))
print(product_number)
print(j)

In [ ]:
unwanted = ["__sort", "ksort", "time1", "time2", "dist", "kindId", "subjectId", "subjectParentId", "brandId",
            "siteBrandId", "supplierId", "logisticsCost", "saleConditions", "panelPromoId", "promoTextCat", "volume", "colors",
            "sizes", "diffPrice", "pics", "isNew", "isAdult"]

for product_number in range(len(data)):
    for key in unwanted:
        data[str(product_number)].pop(key, None)

In [ ]:
while number < len(data):
    time.sleep(1)
    data[str(number)]["sold_quantity"] = show_buys(data[str(number)]["id"])
    number += 1

In [5]:
# Загрузка модели машинного обучения
print("Script started")
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
print("Libraries was upload")
global tokenizer
import os
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
global model
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)
print("Model was upload")

Script started


c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Libraries was upload
Model was upload


In [6]:
# Функция анализа на тональность
def predict_emotion(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    response = torch.nn.functional.softmax(outputs.logits, dim=1)
    stat = response
    response = torch.argmax(response, dim=1).numpy()
    stat = list(stat[0])

    response = list(response)
    if 0 in response:
        response = "Нейтрально"
    elif 1 in response:
        response = "Положительно"
    elif 2 in response:
        response = "Негативно"

    accuracy = None
    if response == "Негативно":
        accuracy = round(stat[2].detach().numpy() * 100, 2)
    elif response == "Положительно":
        accuracy = round(stat[1].detach().numpy() * 100, 2)
    elif response == "Нейтрально":
        accuracy = round(stat[0].detach().numpy() * 100, 2)
    
    if accuracy >= 65: 
        return response
    else:
        return "Нейтрально"

In [164]:
while i < len(data):
    print("="*15)
    print(f"{i}) Отправка запроса на отзывы.")
    feedbacks = get_about_product(str(data[str(i)]["root"])).json()["feedbacks"]
    if feedbacks != None and feedbacks != []:
        texts = []
        answers = []
        photos_list = []

        if len(feedbacks) >= 50:
            m = 0
            while m < 50:
                year_max = 0
                month_max = 0
                day_max = 0
                order_number = 0
                for j in range(len(feedbacks)):
                    year = int(feedbacks[j]["createdDate"][0:4])
                    month = int(feedbacks[j]["createdDate"][5:7])
                    day = int(feedbacks[j]["createdDate"][8:10])
                    if year > year_max:
                        year_max = year
                        month_max = month
                        day_max = day
                        text = feedbacks[j]["text"]
                        order_number = j
                        if feedbacks[j]["answer"] != None:
                            answer = 1
                        else:
                            answer = 0
                        if "photos" in feedbacks[j]:
                            photos = 1
                        else:
                            photos = 0
                    elif year == year_max:
                        if month > month_max:
                            year_max = year
                            month_max = month
                            day_max = day
                            text = feedbacks[j]["text"]
                            order_number = j
                            if feedbacks[j]["answer"] != None:
                                answer = 1
                            else:
                                answer = 0
                            if "photos" in feedbacks[j]:
                                photos = 1
                            else:
                                photos = 0
                        elif month == month_max:
                            if day > day_max:
                                year_max = year
                                month_max = month
                                day_max = day
                                text = feedbacks[j]["text"]
                                order_number = j
                                if feedbacks[j]["answer"] != None:
                                    answer = 1
                                else:
                                    answer = 0
                                if "photos" in feedbacks[j]:
                                    photos = 1
                                else:
                                    photos = 0
                            elif day == day_max:
                                year_max = year
                                month_max = month
                                day_max = day
                                text = feedbacks[j]["text"]
                                order_number = j
                                if feedbacks[j]["answer"] != None:
                                    answer = 1
                                else:
                                    answer = 0
                                if "photos" in feedbacks[j]:
                                    photos = 1
                                else:
                                    photos = 0
                texts.append(text)
                answers.append(answer)
                photos_list.append(photos)
                feedbacks.pop(order_number)
                m += 1
        else:
            for n in range(len(feedbacks)):
                texts.append(feedbacks[n]["text"])
                if feedbacks[n]["answer"] != None:
                    answer = 1
                else:
                    answer = 0
                if "photos" in feedbacks[n]:
                    photos = 1
                else:
                    photos = 0
                answers.append(answer)
                photos_list.append(photos)
        data[str(i)]["neutral_feedbacks"] = 0
        data[str(i)]["positive_feedbacks"] = 0
        data[str(i)]["negative_feedbacks"] = 0
        print("Начинаем ML")
        for j in range(len(texts)):
            result = predict_emotion(texts[j])
            if result == "Нетрально":
                data[str(i)]["neutral_feedbacks"] += 1
            elif result == "Положительно":
                data[str(i)]["positive_feedbacks"] += 1
            else:
                data[str(i)]["negative_feedbacks"] += 1
        data[str(i)]["neutral_feedbacks"] = data[str(i)]["neutral_feedbacks"] / len(texts) * 100
        data[str(i)]["positive_feedbacks"] = data[str(i)]["positive_feedbacks"] / len(texts) * 100
        data[str(i)]["negative_feedbacks"] = data[str(i)]["negative_feedbacks"] / len(texts) * 100
        print("Записали тональность")
        number_of_answers = 0
        for answer in answers:
            if answer == 1:
                number_of_answers += 1
        data[str(i)]["brand_answers"] = number_of_answers / len(texts) * 100

        number_with_photos = 0
        for photo in photos_list:
            if photo == 1:
                number_with_photos += 1
        data[str(i)]["with_photos"] = number_with_photos / len(texts) * 100
    else:
        print("Отзывов нет")
    i += 1

46000) Отправка запроса на отзывы.


c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Отзывов нет
46001) Отправка запроса на отзывы.
Отзывов нет
46002) Отправка запроса на отзывы.


c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Отзывов нет
46003) Отправка запроса на отзывы.
Начинаем ML
Записали тональность
46004) Отправка запроса на отзывы.
Отзывов нет
46005) Отправка запроса на отзывы.


...

c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Начинаем ML
Записали тональность
46107) Отправка запроса на отзывы.


c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Начинаем ML
Записали тональность
46108) Отправка запроса на отзывы.
Отзывов нет


c:\users\rayak\appdata\local\programs\python\python39\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feedbacks2.wb.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [170]:
for i in data:
    data[i]["priceU"] = data[i]["priceU"] / 100
    data[i]["salePriceU"] = data[i]["salePriceU"] / 100

In [173]:
# Проверка кол-ва нейтральных отзывов
a = 0
for i in final_data:
    if "neutral_feedbacks" in final_data[i]:
        if final_data[i]["neutral_feedbacks"] != 0:
            a += 1
print(a)

0


In [174]:
#Их нет. Поэтому удаляю их из данных
for i in final_data:
    if "neutral_feedbacks" in final_data[i]:
        final_data[i].pop("neutral_feedbacks")

In [175]:
json.dump(final_data, open("for_analysis_final.json", "w", encoding="utf-8"), ensure_ascii=False)

In [ ]:
# Анализ

In [ ]:
# Переделываем json в csv

In [176]:
columns = ['id', 'name', 'brand', 'sale', 'priceU', 'salePriceU', 'rating', 'feedbacks', 'category_name', 'subcategory_name',
           'sold_quantity', 'positive_feedbacks', 'negative_feedbacks', 'brand_answers', 'with_photos']

In [177]:
products_list = []
for element in final_data:
    products_list.append(final_data[element])

In [178]:
with open("data.csv", 'w', encoding="utf-8") as file:
    wr = csv.DictWriter(file, fieldnames = columns)
    wr.writeheader()
    wr.writerows(products_list)

In [3]:
df = pd.read_csv('data.csv')

In [4]:
df.head()

,id,name,brand,sale,priceU,salePriceU,rating,feedbacks,category_name,subcategory_name,sold_quantity,positive_feedbacks,negative_feedbacks,brand_answers,with_photos
0,137792600,Блузка праздничная нарядная офисная белая черная,LADY OF THE MOON,78,4850.0,1067.0,5,3612,Блузки и рубашки,NaN,28700,92.0,8.0,86.0,70.0
1,23086008,Рубашка оверсайз,Life in StyIe,78,6000.0,1320.0,5,17605,Брюки,NaN,91700,6.0,94.0,0.0,100.0
2,38709879,Рубашка офисная оверсайз,Darya Selskaya,92,12937.0,1034.0,5,15532,Верхняя одежда,NaN,100000,96.0,4.0,100.0,100.0
3,121011147,Блузка офисная для девочки школьная,Camicia,83,5000.0,850.0,5,7569,"Джемперы, водолазки и кардиганы",NaN,32000,24.0,76.0,0.0,80.0
4,80012708,Рубашка оверсайз базовая,Olesa Chugunova,80,6090.0,1218.0,5,6424,Джинсы,NaN,69500,98.0,2.0,0.0,100.0


In [18]:
print(len(df["category_name"].unique()))
categories = df["category_name"].unique()
categories

26


array(['Блузки и рубашки', 'Брюки', 'Верхняя одежда',
       'Джемперы, водолазки и кардиганы', 'Джинсы', 'Комбинезоны',
       'Костюмы', 'Лонгсливы', 'Пиджаки, жилеты и жакеты',
       'Платья и сарафаны', 'Толстовки, свитшоты и худи', 'Туники',
       'Футболки и топы', 'Халаты', 'Шорты', 'Юбки', 'Белье',
       'Большие размеры', 'Будущие мамы', 'Для высоких', 'Для невысоких',
       'Одежда для дома', 'Офис', 'Пляжная мода', 'Религиозная',
       'Свадьба'], dtype=object)

In [150]:
number_columns = ["sale", "priceU", "salePriceU", "rating", "feedbacks", "sold_quantity", "positive_feedbacks",
                  "negative_feedbacks", "brand_answers", "with_photos"]

In [ ]:
#Меры центральной тенденции

In [185]:
mean_values = {'Variable': ['Скидка', 'Цена без скидки', 'Цена со скидкой', 'Рейтинг', 'Количество отзывов',
                            'Количество продаж', 'Положительные отзывы', 'Отрицательные отзывы',
                            '% отзывов с ответами от бренда', '% отзывов с фотографиями'],
               'Mean': [df['sale'].mean(), df['priceU'].mean(), df['salePriceU'].mean(), df['rating'].mean(),
                        df['feedbacks'].mean(), df['sold_quantity'].mean(), df['positive_feedbacks'].mean(),
                        df['negative_feedbacks'].mean(), df['brand_answers'].mean(), df['with_photos'].mean()]}

for i in range(len(mean_values["Mean"])):
    mean_values["Mean"][i] = round(mean_values["Mean"][i], 2)
    
mean_df = pd.DataFrame(mean_values)
print(mean_df)

                         Variable     Mean
0                          Скидка    52.49
1                 Цена без скидки  4724.90
2                 Цена со скидкой  1961.60
3                         Рейтинг     4.56
4              Количество отзывов  1587.14
5               Количество продаж  6068.18
6            Положительные отзывы    70.70
7            Отрицательные отзывы    32.56
8  % отзывов с ответами от бренда    76.91
9        % отзывов с фотографиями    28.34


In [161]:
pd.set_option('display.max_columns', None)
mean_values_categories = {'Variable': ['Скидка', 'Цена без скидки', 'Цена со скидкой', 'Рейтинг', 'Количество отзывов',
                            'Количество продаж', 'Положительные отзывы', 'Отрицательные отзывы',
                            '% отзывов с ответами от бренда', '% отзывов с фотографиями']}

for i in categories:
    mean_values_categories[i] = []
    for j in range(len(number_columns)):
        mean_values_categories[i].append(round(df[df["category_name"] == i][number_columns[j]].mean(), 2))
    
mean_df_categories = pd.DataFrame(mean_values_categories)
print(mean_df_categories)

                         Variable  Блузки и рубашки     Брюки  Верхняя одежда  \
0                          Скидка             55.06     55.36           56.00   
1                 Цена без скидки           4761.91   4904.50         4571.62   
2                 Цена со скидкой           1876.83   2037.16         1776.88   
3                         Рейтинг              4.86      4.85            4.85   
4              Количество отзывов           4716.03   4480.40         4147.77   
5               Количество продаж          28179.21  24899.21        21931.20   
6            Положительные отзывы             71.83     71.19           71.53   
7            Отрицательные отзывы             27.53     28.74           28.33   
8  % отзывов с ответами от бренда             70.47     71.79           73.66   
9        % отзывов с фотографиями             40.10     33.51           38.00   

   Джемперы, водолазки и кардиганы    Джинсы  Комбинезоны   Костюмы  \
0                            52.91   

In [186]:
median_values = {'Variable': ['Скидка', 'Цена без скидки', 'Цена со скидкой', 'Рейтинг', 'Количество отзывов',
                            'Количество продаж', 'Положительные отзывы', 'Отрицательные отзывы',
                            '% отзывов с ответами от бренда', '% отзывов с фотографиями'],
               'Median': [df['sale'].median(), df['priceU'].median(), df['salePriceU'].median(), df['rating'].median(),
                        df['feedbacks'].median(), df['sold_quantity'].median(), df['positive_feedbacks'].median(),
                        df['negative_feedbacks'].median(), df['brand_answers'].median(), df['with_photos'].median()]}

for i in range(len(median_values["Median"])):
    median_values["Median"][i] = round(median_values["Median"][i])
    
median_df = pd.DataFrame(median_values)
print(median_df)

                         Variable  Median
0                          Скидка      55
1                 Цена без скидки    3150
2                 Цена со скидкой    1339
3                         Рейтинг       5
4              Количество отзывов     259
5               Количество продаж     900
6            Положительные отзывы      70
7            Отрицательные отзывы      30
8  % отзывов с ответами от бренда      96
9        % отзывов с фотографиями      20


In [187]:
median_values_categories = {'Variable': ['Скидка', 'Цена без скидки', 'Цена со скидкой', 'Рейтинг',
                                         'Количество отзывов', 'Количество продаж', 'Положительные отзывы',
                                         'Отрицательные отзывы', '% отзывов с ответами от бренда', '% отзывов с фотографиями']}

for i in categories:
    median_values_categories[i] = []
    for j in range(len(number_columns)):
        median_values_categories[i].append(round(df[df["category_name"] == i][number_columns[j]].median()))
    
median_df_categories = pd.DataFrame(median_values_categories)
print(median_df_categories)

                         Variable  Блузки и рубашки  Брюки  Верхняя одежда  \
0                          Скидка                56     58              58   
1                 Цена без скидки              3290   3340            3290   
2                 Цена со скидкой              1229   1291            1242   
3                         Рейтинг                 5      5               5   
4              Количество отзывов              1808   1394            1304   
5               Количество продаж             14500   9100            7800   
6            Положительные отзывы                74     74              74   
7            Отрицательные отзывы                26     27              26   
8  % отзывов с ответами от бренда                92     94              96   
9        % отзывов с фотографиями                24     22              24   

   Джемперы, водолазки и кардиганы  Джинсы  Комбинезоны  Костюмы  Лонгсливы  \
0                               56      57           58       

In [9]:
# Определение зависимой переменной и независимых переменных
df_without_nan = df.dropna(subset=['positive_feedbacks'])
X = df_without_nan[['sale', 'priceU', 'salePriceU', 'rating', 'feedbacks', 'positive_feedbacks', 'negative_feedbacks',
                    'brand_answers', 'with_photos']]
y = df_without_nan['sold_quantity'] 

# Добавление константы
X = sm.add_constant(X)

# Построение модели множественной линейной регрессии
model = sm.OLS(y, X)
results = model.fit()

# Вывод результатов
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     1928.
Date:                Fri, 19 May 2023   Prob (F-statistic):               0.00
Time:                        13:03:53   Log-Likelihood:            -2.4080e+05
No. Observations:               22354   AIC:                         4.816e+05
Df Residuals:                   22344   BIC:                         4.817e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                485.7197    800

In [10]:
# H1
reviews = df['feedbacks']
sales = df['sold_quantity']

pearson_corr, pearson_p_value = stats.pearsonr(reviews, sales)

print(f'Коэффициент корреляции Пирсона: {round(pearson_corr, 2)}')
print(f'p-значение: {pearson_p_value}')

Коэффициент корреляции Пирсона: 0.65
p-значение: 0.0


In [11]:
# H1a
df_without_nan = df.dropna(subset=['positive_feedbacks'])

positive_reviews = df_without_nan['positive_feedbacks']
sales = df_without_nan['sold_quantity']

corr_coeff, p_value = stats.pearsonr(positive_reviews, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coeff, 2))
print("p-значение:", round(p_value, 2))

Коэффициент корреляции Пирсона: 0.01
p-значение: 0.16


In [63]:
from scipy.stats import spearmanr
df_without_nan = df.dropna(subset=['positive_feedbacks'])

positive_reviews = df_without_nan['positive_feedbacks']
sales = df_without_nan['sold_quantity']

correlation, p_value = spearmanr(positive_reviews, sales)

print("Коэффициент Спирмена:", round(correlation, 2))
print("p-значение:", round(p_value, 2))

Коэффициент Спирмена: 0.02
p-значение: 0.0


In [16]:
# H1b
df_without_nan = df.dropna(subset=['negative_feedbacks'])

negative_reviews = df_without_nan['negative_feedbacks']
sales = df_without_nan['sold_quantity']

corr_coeff, p_value = stats.pearsonr(negative_reviews, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coeff, 2))
print("p-значение:", round(p_value, 2))

Коэффициент корреляции Пирсона: -0.09
p-значение: 0.0


In [37]:
# H1c
for i in df["category_name"].unique():
    reviews_category = df[df["category_name"] == i]["feedbacks"]
    sales_category = df[df["category_name"] == i]["sold_quantity"]
    pearson_corr, pearson_p_value = stats.pearsonr(reviews_category, sales_category)
    print(f'coef = {round(pearson_corr, 2)}; p = {round(pearson_p_value, 3)} - {i}')

coef = 0.79; p = 0.0 - Блузки и рубашки
coef = 0.74; p = 0.0 - Брюки
coef = 0.8; p = 0.0 - Верхняя одежда
coef = 0.78; p = 0.0 - Джемперы, водолазки и кардиганы
coef = 0.77; p = 0.0 - Джинсы
coef = 0.71; p = 0.0 - Комбинезоны
coef = 0.68; p = 0.0 - Костюмы
coef = 0.73; p = 0.0 - Лонгсливы
coef = 0.67; p = 0.0 - Пиджаки, жилеты и жакеты
coef = 0.67; p = 0.0 - Платья и сарафаны
coef = 0.7; p = 0.0 - Толстовки, свитшоты и худи
coef = 0.73; p = 0.0 - Туники
coef = 0.69; p = 0.0 - Футболки и топы
coef = 0.5; p = 0.0 - Халаты
coef = 0.47; p = 0.0 - Шорты
coef = 0.64; p = 0.0 - Юбки
coef = 0.6; p = 0.0 - Белье
coef = 0.55; p = 0.0 - Большие размеры
coef = 0.56; p = 0.0 - Будущие мамы
coef = 0.46; p = 0.0 - Для высоких
coef = 0.57; p = 0.0 - Для невысоких
coef = 0.62; p = 0.0 - Одежда для дома
coef = 0.52; p = 0.0 - Офис
coef = 0.52; p = 0.0 - Пляжная мода
coef = 0.52; p = 0.0 - Религиозная
coef = 0.51; p = 0.0 - Свадьба


In [38]:
# H1d
df_without_nan = df.dropna(subset=['positive_feedbacks'])
for i in df_without_nan["category_name"].unique():
    positive_reviews = df_without_nan[df_without_nan["category_name"] == i]["positive_feedbacks"]
    sales_category = df_without_nan[df_without_nan["category_name"] == i]["sold_quantity"]
    correlation, p_value = spearmanr(positive_reviews, sales_category)
    print(f'coef = {round(correlation, 2)}; p = {round(p_value, 3)} - {i}')

coef = 0.18; p = 0.007 - Блузки и рубашки
coef = 0.11; p = 0.092 - Брюки
coef = 0.05; p = 0.415 - Верхняя одежда
coef = 0.13; p = 0.048 - Джемперы, водолазки и кардиганы
coef = 0.2; p = 0.003 - Джинсы
coef = 0.1; p = 0.108 - Комбинезоны
coef = 0.08; p = 0.26 - Костюмы
coef = 0.01; p = 0.861 - Лонгсливы
coef = 0.02; p = 0.78 - Пиджаки, жилеты и жакеты
coef = 0.03; p = 0.11 - Платья и сарафаны
coef = 0.06; p = 0.346 - Толстовки, свитшоты и худи
coef = -0.03; p = 0.664 - Туники
coef = 0.04; p = 0.589 - Футболки и топы
coef = 0.16; p = 0.011 - Халаты
coef = 0.05; p = 0.46 - Шорты
coef = 0.11; p = 0.113 - Юбки
coef = 0.01; p = 0.478 - Белье
coef = -0.04; p = 0.028 - Большие размеры
coef = -0.01; p = 0.557 - Будущие мамы
coef = 0.01; p = 0.834 - Для высоких
coef = 0.1; p = 0.136 - Для невысоких
coef = -0.03; p = 0.19 - Одежда для дома
coef = -0.03; p = 0.26 - Офис
coef = -0.0; p = 0.962 - Пляжная мода
coef = 0.02; p = 0.713 - Религиозная
coef = -0.0; p = 0.934 - Свадьба


In [45]:
from scipy.stats import kendalltau
df_without_nan = df.dropna(subset=['positive_feedbacks'])
for i in df_without_nan["category_name"].unique():
    if i != "Блузки и рубашки" and i != "Джемперы, водолазки и кардиганы" and i != "Джинсы" and i != "Халаты"\
    and i != "Большие размеры":
        positive_reviews = df_without_nan[df_without_nan["category_name"] == i]["positive_feedbacks"]
        sales_category = df_without_nan[df_without_nan["category_name"] == i]["sold_quantity"]
        correlation, p_value = kendalltau(positive_reviews, sales_category)
        print(f'coef = {round(correlation, 2)}; p = {round(p_value, 3)} - {i}')

coef = 0.08; p = 0.076 - Брюки
coef = 0.04; p = 0.393 - Верхняя одежда
coef = 0.08; p = 0.081 - Комбинезоны
coef = 0.05; p = 0.241 - Костюмы
coef = 0.01; p = 0.875 - Лонгсливы
coef = 0.02; p = 0.709 - Пиджаки, жилеты и жакеты
coef = 0.02; p = 0.07 - Платья и сарафаны
coef = 0.05; p = 0.306 - Толстовки, свитшоты и худи
coef = -0.03; p = 0.58 - Туники
coef = 0.03; p = 0.544 - Футболки и топы
coef = 0.04; p = 0.386 - Шорты
coef = 0.08; p = 0.066 - Юбки
coef = 0.01; p = 0.368 - Белье
coef = -0.01; p = 0.614 - Будущие мамы
coef = 0.01; p = 0.786 - Для высоких
coef = 0.07; p = 0.114 - Для невысоких
coef = -0.02; p = 0.212 - Одежда для дома
coef = -0.01; p = 0.351 - Офис
coef = 0.0; p = 0.995 - Пляжная мода
coef = 0.01; p = 0.743 - Религиозная
coef = -0.0; p = 0.997 - Свадьба


In [46]:
# H1e
df_without_nan = df.dropna(subset=['negative_feedbacks'])
for i in df_without_nan["category_name"].unique():
    negative_reviews = df_without_nan[df_without_nan["category_name"] == i]["negative_feedbacks"]
    sales_category = df_without_nan[df_without_nan["category_name"] == i]["sold_quantity"]
    correlation, p_value = stats.pearsonr(negative_reviews, sales_category)
    print(f'coef = {round(correlation, 2)}; p = {round(p_value, 3)} - {i}')

coef = -0.05; p = 0.477 - Блузки и рубашки
coef = -0.06; p = 0.332 - Брюки
coef = -0.05; p = 0.416 - Верхняя одежда
coef = -0.14; p = 0.044 - Джемперы, водолазки и кардиганы
coef = -0.23; p = 0.0 - Джинсы
coef = -0.12; p = 0.063 - Комбинезоны
coef = -0.1; p = 0.147 - Костюмы
coef = -0.05; p = 0.478 - Лонгсливы
coef = 0.0; p = 0.957 - Пиджаки, жилеты и жакеты
coef = -0.09; p = 0.0 - Платья и сарафаны
coef = -0.03; p = 0.619 - Толстовки, свитшоты и худи
coef = -0.1; p = 0.134 - Туники
coef = -0.15; p = 0.025 - Футболки и топы
coef = -0.19; p = 0.003 - Халаты
coef = -0.08; p = 0.223 - Шорты
coef = -0.08; p = 0.262 - Юбки
coef = -0.11; p = 0.0 - Белье
coef = -0.05; p = 0.004 - Большие размеры
coef = -0.06; p = 0.0 - Будущие мамы
coef = -0.19; p = 0.004 - Для высоких
coef = -0.14; p = 0.031 - Для невысоких
coef = -0.08; p = 0.0 - Одежда для дома
coef = -0.07; p = 0.002 - Офис
coef = -0.11; p = 0.004 - Пляжная мода
coef = -0.1; p = 0.03 - Религиозная
coef = -0.08; p = 0.003 - Свадьба


In [47]:
df_without_nan = df.dropna(subset=['negative_feedbacks'])
for i in df_without_nan["category_name"].unique():
    if i != "Джемперы, водолазки и кардиганы" and i != "Джинсы" and i != "Платья и сарафаны" and i != "Футболки и топы"\
    and i != "Халаты" and i != "Белье" and i != "Большие размеры" and i != "Будущие мамы" and i != "Для высоких"\
    and i != "Для невысоких" and i != "Одежда для дома" and i != "Офис" and i != "Пляжная мода" and i != "Религиозная"\
    and i != "Свадьба":
        negative_reviews = df_without_nan[df_without_nan["category_name"] == i]["negative_feedbacks"]
        sales_category = df_without_nan[df_without_nan["category_name"] == i]["sold_quantity"]
        correlation, p_value = spearmanr(negative_reviews, sales_category)
        print(f'coef = {round(correlation, 2)}; p = {round(p_value, 3)} - {i}')

coef = -0.18; p = 0.006 - Блузки и рубашки
coef = -0.15; p = 0.019 - Брюки
coef = -0.08; p = 0.209 - Верхняя одежда
coef = -0.17; p = 0.007 - Комбинезоны
coef = -0.1; p = 0.14 - Костюмы
coef = -0.08; p = 0.269 - Лонгсливы
coef = -0.11; p = 0.086 - Пиджаки, жилеты и жакеты
coef = -0.17; p = 0.008 - Толстовки, свитшоты и худи
coef = -0.06; p = 0.372 - Туники
coef = -0.15; p = 0.026 - Шорты
coef = -0.19; p = 0.004 - Юбки


In [49]:
df_without_nan = df.dropna(subset=['negative_feedbacks'])
for i in df_without_nan["category_name"].unique():
    if i != "Джемперы, водолазки и кардиганы" and i != "Джинсы" and i != "Платья и сарафаны" and i != "Футболки и топы"\
    and i != "Халаты" and i != "Белье" and i != "Большие размеры" and i != "Будущие мамы" and i != "Для высоких"\
    and i != "Для невысоких" and i != "Одежда для дома" and i != "Офис" and i != "Пляжная мода" and i != "Религиозная"\
    and i != "Свадьба" and i != "Блузки и рубашки" and i != "Брюки" and i != "Комбинезоны" and i !=\
    "Толстовки, свитшоты и худи" and i != "Шорты" and i != "Юбки":
        negative_reviews = df_without_nan[df_without_nan["category_name"] == i]["negative_feedbacks"]
        sales_category = df_without_nan[df_without_nan["category_name"] == i]["sold_quantity"]
        correlation, p_value = kendalltau(negative_reviews, sales_category)
        print(f'coef = {round(correlation, 2)}; p = {round(p_value, 3)} - {i}')

coef = -0.06; p = 0.19 - Верхняя одежда
coef = -0.07; p = 0.129 - Костюмы
coef = -0.05; p = 0.272 - Лонгсливы
coef = -0.08; p = 0.07 - Пиджаки, жилеты и жакеты
coef = -0.04; p = 0.402 - Туники


In [56]:
# H2
from scipy.stats import ttest_ind

with_reviews = df[df['feedbacks'] > 0]['sold_quantity']
without_reviews = df[df['feedbacks'] == 0]['sold_quantity']

t_statistic, p_value = ttest_ind(with_reviews, without_reviews)

print(round(t_statistic, 2))
print(round(p_value, 3))

21.04
0.0


In [62]:
# H2a
from scipy.stats import ttest_ind

for i in df["category_name"].unique():
    df_category = df[df["category_name"] == i]
    with_reviews = df_category[df_category['feedbacks'] > 0]["sold_quantity"]
    without_reviews = df_category[df_category['feedbacks'] == 0]["sold_quantity"]

    t_statistic, p_value = ttest_ind(with_reviews, without_reviews)
    print(f't = {round(t_statistic, 2)}; p = {round(p_value, 3)} - {i}')

C:\Users\rayak\AppData\Local\Temp\ipykernel_11044\1801843518.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_statistic, p_value = ttest_ind(with_reviews, without_reviews)


t = 1.24; p = 0.216 - Блузки и рубашки
t = 1.12; p = 0.265 - Брюки
t = 1.3; p = 0.194 - Верхняя одежда
t = 1.24; p = 0.214 - Джемперы, водолазки и кардиганы
t = 1.37; p = 0.171 - Джинсы
t = 1.97; p = 0.05 - Комбинезоны
t = 1.48; p = 0.141 - Костюмы
t = 1.61; p = 0.109 - Лонгсливы
t = 1.31; p = 0.191 - Пиджаки, жилеты и жакеты
t = 6.49; p = 0.0 - Платья и сарафаны
t = 1.85; p = 0.065 - Толстовки, свитшоты и худи
t = 1.59; p = 0.112 - Туники
t = 2.12; p = 0.035 - Футболки и топы
t = 1.64; p = 0.101 - Халаты
t = 1.65; p = 0.1 - Шорты
t = 1.99; p = 0.047 - Юбки
t = 6.81; p = 0.0 - Белье
t = 7.46; p = 0.0 - Большие размеры
t = 8.31; p = 0.0 - Будущие мамы
t = 2.42; p = 0.016 - Для высоких
t = 1.99; p = 0.047 - Для невысоких
t = 6.53; p = 0.0 - Одежда для дома
t = 6.79; p = 0.0 - Офис
t = 3.3; p = 0.001 - Пляжная мода
t = 2.92; p = 0.004 - Религиозная
t = 5.94; p = 0.0 - Свадьба


In [66]:
# H3
price = df["priceU"]
sales = df["sold_quantity"]

corr_coef, p_value = stats.pearsonr(price, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coef, 2))
print("p-значение:", round(p_value, 3))

Коэффициент корреляции Пирсона: -0.06
p-значение: 0.0


In [68]:
price = df["salePriceU"]
sales = df["sold_quantity"]

corr_coef, p_value = stats.pearsonr(price, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coef, 2))
print("p-значение:", round(p_value, 3))

Коэффициент корреляции Пирсона: -0.1
p-значение: 0.0


In [75]:
# H3a
import statsmodels.api as sm
df_without_nan = df.dropna(subset=['positive_feedbacks'])

y = df_without_nan["sold_quantity"]

X = df_without_nan[["priceU", "positive_feedbacks"]]

X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     68.49
Date:                Fri, 19 May 2023   Prob (F-statistic):           2.21e-30
Time:                        19:21:00   Log-Likelihood:            -2.4716e+05
No. Observations:               22354   AIC:                         4.943e+05
Df Residuals:                   22351   BIC:                         4.943e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               7104.7326    278

In [71]:
import statsmodels.api as sm
df_without_nan = df.dropna(subset=['positive_feedbacks'])

y = df_without_nan["sold_quantity"]

X = df_without_nan[["salePriceU", "positive_feedbacks"]]

X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     132.2
Date:                Fri, 19 May 2023   Prob (F-statistic):           8.42e-58
Time:                        19:17:25   Log-Likelihood:            -2.4709e+05
No. Observations:               22354   AIC:                         4.942e+05
Df Residuals:                   22351   BIC:                         4.942e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               7354.7238    276

In [72]:
# H3b
import statsmodels.api as sm
df_without_nan = df.dropna(subset=['negative_feedbacks'])

y = df_without_nan["sold_quantity"]

X = df_without_nan[["priceU", "negative_feedbacks"]]

X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     162.5
Date:                Fri, 19 May 2023   Prob (F-statistic):           8.48e-71
Time:                        19:19:27   Log-Likelihood:            -2.4706e+05
No. Observations:               22354   AIC:                         4.941e+05
Df Residuals:                   22351   BIC:                         4.942e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               9696.5783    207

In [76]:
import statsmodels.api as sm
df_without_nan = df.dropna(subset=['negative_feedbacks'])

y = df_without_nan["sold_quantity"]

X = df_without_nan[["salePriceU", "negative_feedbacks"]]

X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     219.4
Date:                Fri, 19 May 2023   Prob (F-statistic):           4.44e-95
Time:                        19:23:01   Log-Likelihood:            -2.4701e+05
No. Observations:               22354   AIC:                         4.940e+05
Df Residuals:                   22351   BIC:                         4.940e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               9781.8038    204

In [78]:
# H4
rating = df["rating"]
sales = df["sold_quantity"]

corr_coef, p_value = stats.pearsonr(rating, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coef, 2))
print("p-значение:", round(p_value, 3))

Коэффициент корреляции Пирсона: 0.12
p-значение: 0.0


In [80]:
# H4a
filtered_df_group_1 = df[(df['rating'] >= 4) & (df['rating'] <= 4.7)]
filtered_df_group_2 = df[(df['rating'] > 4.7)]

group_1 = filtered_df_group_1['sold_quantity']
group_2 = filtered_df_group_2['sold_quantity']

t_statistic, p_value = stats.ttest_ind(group_1, group_2, equal_var=False)

print(f"t-статистика: {round(t_statistic, 2)}")
print(f"p-значение: {round(p_value, 3)}")

t-статистика: -20.01
p-значение: 0.0


In [82]:
# H5
df_without_nan = df.dropna(subset=['brand_answers'])
brand_answers = df_without_nan["brand_answers"]
sales = df_without_nan["sold_quantity"]

corr_coef, p_value = stats.pearsonr(brand_answers, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coef, 2))
print("p-значение:", round(p_value, 3))

Коэффициент корреляции Пирсона: -0.12
p-значение: 0.0


In [86]:
# H5a
df_without_nan = df.dropna(subset=['brand_answers'])

X = df_without_nan[['brand_answers', 'negative_feedbacks']]

X = sm.add_constant(X)

y = df_without_nan['sold_quantity'] 

model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:          sold_quantity   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     308.9
Date:                Fri, 19 May 2023   Prob (F-statistic):           1.10e-68
Time:                        20:28:32   Log-Likelihood:            -2.4707e+05
No. Observations:               22354   AIC:                         4.941e+05
Df Residuals:                   22352   BIC:                         4.942e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          9588.2152    197.849     48.462

In [89]:
# H6
df_without_nan = df.dropna(subset=['with_photos'])
with_photos = df_without_nan["with_photos"]
sales = df_without_nan["sold_quantity"]

corr_coef, p_value = stats.pearsonr(with_photos, sales)

print("Коэффициент корреляции Пирсона:", round(corr_coef, 2))
print("p-значение:", round(p_value, 3))

Коэффициент корреляции Пирсона: 0.27
p-значение: 0.0


In [91]:
df_without_nan = df.dropna(subset=['with_photos'])

for i in df_without_nan["category_name"].unique():
    df_category = df_without_nan[df_without_nan["category_name"] == i]
    with_photos = df_category["with_photos"]
    sales = df_category["sold_quantity"]

    corr_coef, p_value = stats.pearsonr(with_photos, sales)
    print(f'coef = {round(corr_coef, 2)}; p = {round(p_value, 3)} - {i}')

coef = 0.43; p = 0.0 - Блузки и рубашки
coef = 0.44; p = 0.0 - Брюки
coef = 0.58; p = 0.0 - Верхняя одежда
coef = 0.42; p = 0.0 - Джемперы, водолазки и кардиганы
coef = 0.53; p = 0.0 - Джинсы
coef = 0.45; p = 0.0 - Комбинезоны
coef = 0.41; p = 0.0 - Костюмы
coef = 0.36; p = 0.0 - Лонгсливы
coef = 0.48; p = 0.0 - Пиджаки, жилеты и жакеты
coef = 0.32; p = 0.0 - Платья и сарафаны
coef = 0.43; p = 0.0 - Толстовки, свитшоты и худи
coef = 0.35; p = 0.0 - Туники
coef = 0.34; p = 0.0 - Футболки и топы
coef = 0.09; p = 0.153 - Халаты
coef = 0.1; p = 0.15 - Шорты
coef = 0.19; p = 0.004 - Юбки
coef = 0.23; p = 0.0 - Белье
coef = 0.21; p = 0.0 - Большие размеры
coef = 0.14; p = 0.0 - Будущие мамы
coef = 0.24; p = 0.0 - Для высоких
coef = 0.08; p = 0.213 - Для невысоких
coef = 0.14; p = 0.0 - Одежда для дома
coef = 0.11; p = 0.0 - Офис
coef = 0.06; p = 0.13 - Пляжная мода
coef = 0.2; p = 0.0 - Религиозная
coef = 0.08; p = 0.003 - Свадьба
